# Testing Algorithm


#### Further References:


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

import pm4py


sys.path.append("/app/backend")

from compositional_algorithm.combine_nets.combine_nets import MergeNets
from compositional_algorithm.compositional_algorithm import compositional_discovery
from compositional_algorithm.compositional_algorithm import discover
from compositional_algorithm.compositional_algorithm import generate_unique_id
from compositional_algorithm.compositional_algorithm import is_isomorphic
from compositional_algorithm.compositional_algorithm import is_net_valid
from compositional_algorithm.compositional_algorithm import is_refinement
from compositional_algorithm.compositional_algorithm import priority_identifier
from compositional_algorithm.compositional_algorithm import standardize_properties_log
from compositional_algorithm.interface_patterns.interface_patterns import (
    INTERFACE_PATTERNS,
)
from compositional_algorithm.transformations.transformations import TRANSFORMATIONS
from compositional_algorithm.transformations.transformations import PlaceTransformation
from compositional_algorithm.transformations.transformations import (
    TransitionTransformation,
)
from entropy_conformance.entropy_conformance import entropy_conformance

#### 0. Algorithm Input


In [ ]:
# define input path
log_path = "/app/backend/data_catalog/final_logs/IP-11_initial_log.xes"

df_log = pm4py.read_xes(log_path)
df_log = standardize_properties_log(df_log)
display(df_log.head(5))

# select algorithm
algorithm = pm4py.discover_petri_net_inductive
algorithm_kwargs = {"noise_threshold": 0}
# algorithm = split_miner  # noqa: ERA001

In [4]:
# interface pattern
IP1 = INTERFACE_PATTERNS[0]
ip1_net_a1, ip1_initial_marking_a1, ip1_final_marking_a1 = IP1.get_net("A1")
# pm4py.view_petri_net(ip1_net_a1, ip1_initial_marking_a1, ip1_final_marking_a1) # noqa: ERA001

ip1_net_a2, ip1_initial_marking_a2, ip1_final_marking_a2 = IP1.get_net("A2")
# pm4py.view_petri_net(ip1_net_a2, ip1_initial_marking_a2, ip1_final_marking_a2) # noqa: ERA001

# merge nets
ip1_net = MergeNets.merge_nets([ip1_net_a1, ip1_net_a2])

# add markings
initial_marking, final_marking = MergeNets.add_markings(ip1_net)
# pm4py.view_petri_net(ip1_net, initial_marking, final_marking) # noqa: ERA001

In [5]:
# interface pattern
IP4 = INTERFACE_PATTERNS[2]
ip4_net_a1, ip4_initial_marking_a1, ip4_final_marking_a1 = IP4.get_net("A1")
# pm4py.view_petri_net(ip4_net_a1, ip4_initial_marking_a1, ip4_final_marking_a1)  # noqa: ERA001

ip4_net_a2, ip4_initial_marking_a2, ip4_final_marking_a2 = IP4.get_net("A2")
# pm4py.view_petri_net(ip4_net_a2, ip4_initial_marking_a2, ip4_final_marking_a2)  # noqa: ERA001

# merge nets
ip4_net = MergeNets.merge_nets([ip4_net_a1, ip4_net_a2])

# add markings
initial_marking, final_marking = MergeNets.add_markings(ip4_net)
# pm4py.view_petri_net(ip4_net, initial_marking, final_marking)  # noqa: ERA001


#### 1. Directly Discover with the given input


In [ ]:
# discover by selected algorithm
net, initial_marking, final_marking = discover(log_path, algorithm)
pm4py.view_petri_net(net, initial_marking, final_marking, format="png")

In [ ]:
# TODO: discover for each agent
unique_agents = df_log["org:resource"].unique()
print(unique_agents)  # noqa: T201
combined_nets = []
for agent in unique_agents:
    df = df_log[df_log["org:resource"] == agent]
    # display concept name if s
    display(df[df["concept:name"] == "s"])
    display(df["concept:name"].unique())

    temp_net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(df)
    pm4py.view_petri_net(temp_net, initial_marking, final_marking)
    combined_nets.append(temp_net)

In [ ]:
final_net = MergeNets.merge_nets(combined_nets)

# add markings
initial_marking, final_marking = MergeNets.add_markings(final_net)
pm4py.view_petri_net(final_net, initial_marking, final_marking)

In [ ]:
raise Exception("stop here")

#### 2. Check Net is valid


In [23]:
# check helper functions works
assert is_net_valid(net, net) is True

#### 3. Check Nets are equal


In [24]:
# check net comparison works
assert is_isomorphic(net, net) is True

#### 4. Check ID Hash


In [25]:
assert generate_unique_id(net) == generate_unique_id(net)

#### 5. Check is Refinement


##### 5.1 Apply single refinement


In [ ]:
# sequence
transformation_sequence = []

# initial net. Cpy needed for comparison here
copy_refine_net = net.__deepcopy__()

# apply a P1 transformation
places = list(copy_refine_net.places)
P1 = TRANSFORMATIONS[0]
single_refined_net = P1.refine(places[2], copy_refine_net)
transformation_seq_element = (P1, places[2])
transformation_sequence.append(transformation_seq_element)

print(f"refining at place {places[2]}")  # noqa: T201
print(transformation_sequence)  # noqa: T201
pm4py.view_petri_net(single_refined_net, initial_marking, final_marking, format="png")

In [ ]:
print(priority_identifier(net, net, []))  # noqa: T201
print(priority_identifier(net, single_refined_net, transformation_sequence))  # noqa: T201


##### 5.2 Apply multiple refinements


In [ ]:
# sequence
transformation_sequence = []

# Make a deep copy of the original net. py needed for comparison here
multiple_refined_net = ip1_net_a1.__deepcopy__()

# split in place and transition transformations
place_transformations = [
    t for t in TRANSFORMATIONS if isinstance(t(), PlaceTransformation)
]
transition_transformations = [
    t for t in TRANSFORMATIONS if isinstance(t(), TransitionTransformation)
]

# Note: branching logic: we need to apply all possible transformations
# for each place in the current net
for place in multiple_refined_net.places:
    # apply each possible place transformation
    for place_transformation in place_transformations:
        # Note: Deep copy of the current net before applying the transformation -> transformation change places & transitions and sets are immutable.
        net_copy = multiple_refined_net.__deepcopy__()
        transformed_net = place_transformation.refine(place, net_copy)
        transformation_seq_element = (place_transformation, place)
        transformation_sequence.append(transformation_seq_element)

        # Update multiple_refined_net to the latest transformed net
        multiple_refined_net = transformed_net

# for each transition in the current net
for transition in multiple_refined_net.transitions:
    # apply each possible transition transformation
    for transition_transformation in transition_transformations:
        # Note: ensures that subsequent transformations are applied to a fresh instance of the net.
        net_copy = multiple_refined_net.__deepcopy__()
        transformed_net = transition_transformation.refine(
            transition,
            net_copy,
        )
        transformation_seq_element = (transition_transformation, transition)
        transformation_sequence.append(transformation_seq_element)

    # Update multiple_refined_net to the latest transformed net
    multiple_refined_net = transformed_net

print(transformation_sequence)  # noqa: T201
pm4py.view_petri_net(multiple_refined_net, initial_marking, final_marking, format="png")

In [ ]:
print(priority_identifier(net, multiple_refined_net, transformation_sequence))

In [ ]:
# check if the net is a refinement
is_ref, path = is_refinement(net, net, TRANSFORMATIONS)
assert is_ref is True

In [ ]:
assert not is_isomorphic(net, single_refined_net)

# check refinement algorithm
is_ref, path = is_refinement(net, single_refined_net, TRANSFORMATIONS)
print("Output:", is_ref, path)  # noqa: T201

In [ ]:
raise ValueError("Very Time intensive.")

In [ ]:
# call smalll net with ip3:
net, initial_marking, final_marking = compositional_discovery(
    input_log_path="/app/backend/data_catalog/final_logs/A4_collectivelog.xes",
    algorithm=algorithm,
    interface_pattern=INTERFACE_PATTERNS[
        2
    ],  # or also INTERFACE_PATTERNS for all possible
    transformations=TRANSFORMATIONS,
    agent_column="org:resource",
    **algorithm_kwargs,
)
pm4py.view_petri_net(net, initial_marking, final_marking, format="png")

In [ ]:
# call smalll net with ip4
net, initial_marking, final_marking = compositional_discovery(
    input_log_path="/app/backend/data_catalog/final_logs/R1_collectivelogs.xes",
    algorithm=algorithm,
    interface_pattern=INTERFACE_PATTERNS[
        3
    ],  # or also INTERFACE_PATTERNS for all possible
    transformations=TRANSFORMATIONS,
    agent_column="org:resource",
    **algorithm_kwargs,
)
pm4py.view_petri_net(net, initial_marking, final_marking, format="png")

In [ ]:
assert not is_isomorphic(net, multiple_refined_net)

# check refinement algorithm
is_ref, path = is_refinement(ip1_net_a1, multiple_refined_net, TRANSFORMATIONS)
print("Output:", is_ref, path)  # noqa: T201

In [ ]:
# Check if refinement is possible
is_ref, path = is_refinement(ip1_net_a1, net, TRANSFORMATIONS)
print(is_ref, path)

#### 4. Check Algorithm


In [ ]:
net, initial_marking, final_marking = compositional_discovery(
    input_log_path=log_path,
    algorithm=algorithm,
    interface_pattern=INTERFACE_PATTERNS[
        0
    ],  # or also INTERFACE_PATTERNS for all possible
    transformations=TRANSFORMATIONS,
    agent_column="org:resource",
    **algorithm_kwargs,
)
pm4py.view_petri_net(net, initial_marking, final_marking, format="png")

#### 5. Conformance Checking


In [ ]:
raise ValueError("Very Time intensive.")

In [ ]:
# alignment based fitness and precision
pm4py.fitness_alignments(df_log, net, initial_marking, final_marking)
pm4py.precision_alignments(df_log, net, initial_marking, final_marking)

In [ ]:
# entropy based fitness and precision
precision, recall = entropy_conformance(
    "/app/backend/data_catalog/compositional_process_discovery_experiment_data/IP-1/IP-1_initial_log.xes",
    "/app/backend/data_catalog/compositional_process_discovery_experiment_data/IP-1/IP-1_directly_mined.pnml",
)